In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import base64
from IPython.display import display, Image, Markdown
import pandas as pd
import json

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


## Perform OCR and transform to images

In [3]:
# Import the libraries
from pdf2image import convert_from_path
# Create a function to convert PDF to images and store in specific path
def pdf_to_images(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Convert pdf into images
    images = convert_from_path(pdf_path)
    image_paths = []

    # Save images and paths
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f"page{i+1}.jpg")
        image.save(image_path, "JPEG")
        image_paths.append(image_path)
    return image_paths

In [4]:
pdf_path = "./resources/pdf/Things mother used to make.pdf"
output_folder = "./resources/output_images"
image_paths = pdf_to_images(pdf_path, output_folder)

In [5]:
# Set up connection to openai API

client = OpenAI(
    api_key=api_key
)
model = "gpt-4o-mini"

In [7]:
# Read and encode one image
image_path = "./resources/output_images/page23.jpg"
with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

In [8]:
# Define the system prompt
system_prompt = """
Please analyze the content of this image and extract any related recipe information.
"""

In [9]:
# Call the OpenAI API use the chat completion method
response = client.chat.completions.create(
    model = model,
    messages = [
        # Provide the system prompt
        {"role": "system", "content": system_prompt},

        # The user message contains both the text and image URL / path
        {"role": "user", "content": [
            "This is the image from the recipe page.",
            {"type": "image_url",
             "image_url": {"url": f"data:image/jpeg;base64,{image_data}",
                           "detail": "low"}}
        ]}
    ]
)

In [10]:
# Define a function to get the gpt-response and display in markdown
def display_gpt_response(response):
    gpt_response = response.choices[0].message.content
    return display(Markdown(gpt_response))


In [11]:
display_gpt_response(response)

Here’s the recipe information based on the image you provided:

### Things Mother Used to Make
#### Breads

**Bannocks**
- **Ingredients:**
  - 1 Cupful of Thick Sour Milk
  - ½ Cupful of Sugar
  - 2 Cupfuls of Flour
  - ½ Cupful of Indian Meal
  - 1 Teaspoonful of Soda
  - A pinch of Salt

- **Instructions:**
  1. Make the mixture stiff enough to drop from a spoon.
  2. Drop mixture, size of a walnut, into boiling fat.
  3. Serve warm, with maple syrup.

---

**Boston Brown Bread**
- **Ingredients:**
  - 1 Cupful of Rye Meal
  - 1 Cupful of Graham Meal
  - 1 Cupful of Molasses
  - 1 Cupful of Flour
  - 1 Cupful of Sweet Milk
  - 1 Cupful of Sour Milk
  - ½ Teaspoonful of Soda
  - 1 Heaping Teaspoonful of Salt

- **Instructions:**
  1. Stir the meals and salt together.
  2. Beat the soda into the molasses until it foams; add sour milk, mix well, and pour into a tin cup which has been well greased.
  3. If you have none, use a brown-bread steamer. 

Feel free to ask if you need more information or additional recipes!

In [13]:
# Define improved system prompt
system_prompt2 = """
Please analyze the content of this image and extract any related recipe information into structure components.
Specifically, extra the recipe title, list of ingredients, step by step instructions, cuisine type, dish type, any relevant tags or metadata.
The output must be formatted in a way suited for embedding in a Retrieval Augmented Generation (RAG) system.
"""